In [ ]:
import os
import glob
import joblib
import pickle
import cv2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#brain packages
import nibabel as nb
import nilearn as nil
from nilearn import plotting
from nltools import Brain_Data
from nltools import Design_Matrix

In [ ]:
datapath='/Volumes/Scraplab/data/ds002837/derivatives/'

#Generate the subject list and filenames
func_data = os.listdir(datapath)
sub_ids = [x for x in func_data if ('sub-') in x] #grab all the subject IDs for easy filtering
sub_ids.sort() #sort it numerically

all_task_subs = [] #net together all the datafiles independent of which task they are from
for id in sub_ids:
    all_task_subs.append(glob.glob(os.path.join(datapath+id+'/func/*blur_censor_ica.nii.gz'))[0])


### Video-to-Task Naming Dictionary

First, I reconcile the difference between the subject task names and the naming convention of the videos themselves by creating a Python dictionary:

In [ ]:
#keys
tasknames = ['12yearsaslave','500daysofsummer','backtothefuture','citizenfour',
           'littlemisssunshine', 'pulpfiction','split','theprestige',
           'theshawshankredemption','theusualsuspects']
#values
vidnames = ['12_years_a_slave','500_days_of_summer','back_to_the_future','citizenfour',
           'little_miss_sunshine', 'pulp_fiction','split','the_prestige',
           'the_shawshank_redemption','the_usual_suspects']

zippedlist = zip(tasknames,vidnames)
tasktovidmap = dict(zippedlist)
#task is a key, video is a value

# Generating our Design Matrix

In our design matrix, we include body pose annotations extracted from 10 hollywood movies (learn more about the movies and the data set at the [Naturalistic Neuroimaging Database project page](https://www.naturalistic-neuroimaging-database.org/) using a neural network called [PARE](https://github.com/mkocabas/PARE).

Using those pose annotations, we are interested in calculating several regressors: person prescence, static synchrony, phasic synchrony, aphasic synchrony.

**Person presence** is calculated using the 'frame_ids' value in the PARE output; this tell sus the exact frames where the presence of a person (or several people) was detected on screen.

# Extracting Person Tracking Information from PARE

For this analysis, we are interested in modeling how the number of people on screen might drive neural activity. Since there can be some cases where there is variation in the number of people are on screen at a time, we take the median across the frames that fall within the 1-second TR range.

In [ ]:
### THIS BLOCK OF FUNCTIONS WRITTEN BY LANDRY BULLS ###
def pkl_to_array(pickle_path, video_path):
    data = dict(joblib.load(pickle_path))
    n_tracks = len(data)
    video = cv2.VideoCapture(video_path)
    frameCount = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    arr = np.zeros((n_tracks, frameCount))
    track_ids = list(data.keys())
    # track IDs are not always incremental by 1 and are not in order, so I use enumerate here
    for t, track in enumerate(track_ids):
        # Here, data.get(track) returns the second-level dictionary object for that specific track. I run get() on that
        # as well because it is also a dictionary, which will return an array of frame IDs for that track. 
        frameIDs = data.get(track).get('frame_ids')
        # Here I locate the row t (which corresponds to whatever iteration I'm on in my for loop) and I set all
        # indices that correspond to frame IDs in that track to 1.
        arr[t][frameIDs]=1
    return arr

def collapse(pickle_array):
    frames = pickle_array.shape[1]
    regr = np.zeros((frames))
    for frame in range(frames):
        # Here, we are iterating through each frame (each column at a time) and taking the sum of that column, 
        # then placing that number at the frame index in our output array. 
        regr[frame] = sum(pickle_array[:,frame])
    return regr

def tr_resample(regr_arr, vid_cap, TR, func_length, method='median'):
    #all timestamps are in milliseconds
    fps = float(vid_cap.get(cv2.CAP_PROP_FPS))
    framecount = int(vid_cap.get(cv2.CAP_PROP_FRAME_COUNT))
    video_timestamps = [(1/fps)*1000*frame for frame in range(framecount)]
    brain_timestamps = [TR*1000*tr for tr in range(func_length)]
    if method=='mode':
        func = lambda x: scipy.stats.mode(x, nan_policy='omit')[0]
    elif method=='average':
        func = lambda x: np.nanmean(x)
    elif method=='min':
        func = np.nanmin 
    elif method=='max':
        func = np.nanmax 
    elif method=='median':
        func = np.nanmedian
    out = []
    def find_nearest(array, value):
        array = np.asarray(array)
        idx = (np.abs(array - value)).argmin()
        return idx
    for stamp in range(func_length):
        in_time = brain_timestamps[stamp]
        out_time = in_time+1000*TR
        in_frame, out_frame = find_nearest(video_timestamps, in_time), find_nearest(video_timestamps, out_time)
        regr_frames = regr_arr[in_frame:out_frame]
        out.append(func(regr_frames))
    return np.array(out)

In [ ]:
datapath = '/Volumes/Scraplab/data/ds002837/derivatives/'
movie_lengths = []
for task in tasknames:
    task_files = [x for x in all_task_subs if task in x]
    task_sub_ids = [x.split("/")[6] for x in task_files if ("sub") in x] # this is going to be 9 on Discovery
    
    for sub in task_sub_ids[0:1]:
        task_sub = nb.load(datapath+sub+os.sep+'func'+os.sep+sub+"_task-"+task+"_bold_blur_censor.nii.gz")
        movie_lengths.append(task_sub.shape[3])

movie_dict = dict(zip(vidnames, movie_lengths))

# Person presence regressors

In [ ]:
for video in vidnames:
    vidpath = '/Volumes/Scraplab/stimuli/'+video+'.mp4'
    pklpath = '/Volumes/Scraplab/data/psypose_outs/psypose_pare_nndb_outs/'+video+'_data/psypose_bodies.pkl'
    regr_arr = pkl_to_array(pklpath, vidpath)
    regr_arr = collapse(pkl_arr)
    vid_cap = cv2.VideoCapture(vidpath)
    n_tr = movie_dict[video]
    resamp_arr = tr_resample(regr_arr, vid_cap, 1, n_tr)
    resamp_df = pd.DataFrame(resamp_arr)
    print("movie name: ", video, "resampled array shape:", resamp_arr.shape, 'resampled df shape:', resamp_df.shape, "length of TR: ", n_tr)
    resamp_df.to_csv(video+"_regressor.csv", index=False)
    cv2.destroyAllWindows()

In [ ]:
# separate it out by # of people on screen

for vid in vidnames:
    df = pd.read_csv('glm_analysis/glm_regressors/'+vid+'_regressor.csv')
    df.columns = ['People']
    df[["Zero","One","Two","Three","Four+"]] = ""
    df['Zero'], df['One'], df['Two'], df['Three'], df['Four+'] = np.where(df['People']==0, 1, 0),np.where(df['People']==1, 1, 0), np.where(df['People']==2, 1, 0), np.where(df['People']==3, 1, 0), np.where(df['People']>3, 1, 0)
    df.to_csv('glm_analysis/glm_regressors/'+vid+"_person_presence_dm.csv", index=False)


# Static Synchrony Measures

static_path = '/Volumes/Scraplab/data/synchrony_vectors/static/'

In [ ]:
for task in tasknames:
    video = tasktovidmap[task]
    vidpath = '/Volumes/Scraplab/stimuli/'+video+'.mp4'
    regr_arr = np.load(static_path+video+"__synchrony.npy")
    regr_arr = np.nan_to_num(regr_arr)
    vid_cap = cv2.VideoCapture(vidpath)
    n_tr = movie_dict[video]
    resamp_arr = tr_resample(regr_arr, vid_cap, 1, n_tr)
    resamp_df = pd.DataFrame(resamp_arr)
    print("movie name: ", task,video, "resampled array shape:", resamp_arr.shape, 'resampled df shape:', resamp_df.shape, "length of TR: ", n_tr)
    resamp_df.to_csv(video+"_static_sync.csv", index=False)
    cv2.destroyAllWindows()

# Netting all the regressors together

In [ ]:
for task in tasknames:
    video = tasktovidmap[task]
    vid_regr = pd.read_csv('glm_analysis/glm_regressors/'+video+'_person_presence_dm.csv')
    vid_regr.drop('People',axis=1,inplace=True)
    sync_regr = pd.read_csv(video+'_static_sync.csv')
    regressors = vid_regr.copy()
    regressors["static"] = sync_regr["0"]
    regressors.to_csv('glm_analysis/glm_regressors/'+video+"_full_dm.csv", index=False)

# Running the GLM

In [ ]:
#local version

TR = 1

for task in tasknames[0:1]:
    task_files = [x for x in all_task_subs if task in x]
    task_sub_ids = [x.split("/")[6] for x in task_files if ("sub") in x] # this is going to be 9 on Discovery

    for sub in task_sub_ids[0:1]:
        #First, load subject's functional data
        subj_brain = Brain_Data(glob.glob(os.path.join(datapath+sub+'/func/*blur_censor.nii.gz'))[0])

        #Next, because of the naming mis-match, we grab the subject's task name from the first file to map it to the video
        video = "".join([val for key,val in tasktovidmap.items() if key in subj_brain])

        #Now we can load that task video and convolve it
        df = pd.read_csv('glm_analysis/glm_regressors/'+video+'_full_dm.csv')
        dm = Design_Matrix(df, sampling_freq=1./TR)
        dm = dm.convolve()
        dm = dm.add_poly(order=0) #this is the intercept, we don't need to add higher order polynomials (eg quadratics etc) because NNDb handled it already

        #We ensure that any NaNs in our design matrix are filled, and remove columns with duplicate data 
        dm_cleaned = dm.clean(verbose=True)

        #Set the design matrix, full_dm_cleaned, to the X attribute of the brain data object (subj_run_data)
        subj_brain.X = dm_cleaned
        stats = subj_brain.regress()
        #write our results to a beta map nii file. 
        stats['beta'].write('glm_analysis/'+sub+'_'+video+'_betamap.nii.gz')